# Init

In [1]:
import json
import math
import re

import PyPDF2
import anthropic
import ebooklib
from bs4 import BeautifulSoup
from dotenv import load_dotenv
from ebooklib import epub

In [2]:
model = "claude-3-7-sonnet-latest"

In [3]:
load_dotenv()
client = anthropic.Anthropic()

# Prompts

## Splitted text prompt

### Format

In [4]:
split_text_format = '''{
    question: str,
    textPart: str,
    answers: [
        {
            answer: str,
            isCorrect: boolean,
        }
    ]
}'''

In [5]:
topics = '''1. questions about the personalities of the characters
2. questions about the appearance or look of characters
3. what the characters did in certain situations
4. what the characters did in certain situations, what they thought or felt
5. the attitude of some characters to other characters
6. names of cities or locations where the events took place
7. what are the key characters'''

### Prompt

In [6]:
split_text_prompt = '''Your task is to read a part of the book carefully and create only ONE question for the children who have read this part. 
Question should have 4 answer choices with 1 correct answer.

Here are some topics to create a question:
1. questions about the personalities of the characters
2. questions about the appearance or look of characters
3. what the characters did in certain situations
4. what the characters did in certain situations, what they thought or felt
5. the attitude of some characters to other characters
6. names of cities or locations where the events took place
7. what are the key characters

Try to make question and answer choices more similar to the text itself.
Avoid creating tricky questions.

The question must be in this json format:
```
{}
```

The textPart field of the question should contain a part of the text from which you took the question

Here is a part of a book:
{}

The question must be in the same language as the book
'''

In [7]:
gen_prompt_template = '''You are an experienced educational content creator specializing in reading comprehension exercises for children. Your task is to create a single multiple-choice question based on a given book excerpt.

Here's the book excerpt you'll be working with:

<book_excerpt>
{}
</book_excerpt>

Now, let's review the list of topics that may be relevant to the question you'll create:

<topics_list>
{}
</topics_list>

Your task is to carefully read this excerpt and create ONE question for children who have read this part. Follow these guidelines:

1. Create a question that is closely related to the content of the text.
2. Generate 4 answer choices, with only 1 correct answer.
3. Ensure that the question and answer choices are similar in language and style to the text itself.
4. Avoid creating tricky or intentionally misleading questions or answers.
5. The question must be in the same language as the book excerpt.

You must format your output as a JSON object. Here's the required format:

<json_format>
{}
</json_format>

Important: The "textPart" field of the question should contain the specific part of the text from which you derived the question.

Before providing your final answer, wrap your thought process inside question_development tags. Consider the following steps:

1. Write down 2-3 key quotes from the excerpt that could be potential question sources.
2. List out 3-4 potential topics from the provided topic list that relate to these quotes.
3. For each potential question:
   - Formulate a clear and concise question.
   - Write down the correct answer and three plausible but incorrect answers.
   - Evaluate the question based on clarity, relevance, and difficulty level.
4. Choose the best question based on your evaluation.
5. Select the relevant text part that corresponds to your chosen question.

After your thought process, provide the final JSON output with your question, answers, and relevant text part.'''

In [8]:
gen_prompt_template_2 = '''You are an experienced educational content creator specializing in reading comprehension exercises for children. Your task is to create a single, clear, and specific multiple-choice question based on a given book excerpt.

First, carefully read the following book excerpt:

<book_excerpt>
{}
</book_excerpt>

Now, review this list of topics that may be relevant to the question you'll create:

<topics_list>
{}
</topics_list>

Your goal is to create ONE question for children who have read this excerpt. Follow these guidelines:

1. The question must be closely related to the content of the text.
2. Generate 4 answer choices, with only 1 correct answer.
3. Use language and style similar to the text itself.
4. Avoid creating tricky or intentionally misleading questions or answers.
5. The question must be in the same language as the book excerpt.
6. Ensure the question is clear and specific, explicitly mentioning any relevant context from the excerpt.

Before creating your final question, wrap your reasoning process in <question_development> tags. Follow these steps:

1. Identify 2-3 key quotes from the excerpt that could be potential question sources. Write these quotes down verbatim.
2. List 3-4 relevant topics from the provided topic list that relate to these quotes.
3. For each potential question:
   - Formulate a clear and concise question, ensuring it includes specific context.
   - Write the correct answer and three plausible but incorrect answers.
   - Evaluate the question based on clarity, relevance, and difficulty level.
   - Consider the age-appropriateness of the question for children.
   - Assess how well the question aligns with the provided topics.
4. Choose the best question based on your evaluation.
5. Select the relevant text part that corresponds to your chosen question.
6. Explain why the chosen question is the best option, considering all factors evaluated.

After your question development process, provide the final output as a JSON object in <json_format> wrapper. Here's the required format:

<json_format>
{}
</json_format>

Remember, the "textPart" field should contain the specific part of the text from which you derived the question.'''

# Utils

## Get formatted splitted text prompt

In [9]:
def get_split_text_prompt(text_part: str):
    return split_text_prompt.format(split_text_format, text_part)

In [10]:
def get_gen_prompt(text_part: str):
    return gen_prompt_template_2.format(text_part, topics, split_text_format)

## Remove new lines

In [11]:
def remove_extra_newlines(text):
    return re.sub(r'\n+', '\n', text)

## Read PDF

In [12]:
def read_pdf(file_path: str):
    try:
        with open(file_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            text = ""
            for page_num in range(len(reader.pages)):
                page = reader.pages[page_num]
                text += page.extract_text()
            return text
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

## Read EPUB

In [13]:
def read_epub(file_path):
    book = epub.read_epub(file_path)
    text_content = []

    for item in book.get_items():
        if item.get_type() == ebooklib.ITEM_DOCUMENT:
            soup = BeautifulSoup(item.get_body_content(), 'html.parser')
            text_content.append(soup.get_text())

    text = '\n'.join(text_content)
    return remove_extra_newlines(text)

## Split text

In [14]:
def split_text_into_parts(text: str, num_parts: int):
    paragraphs = re.split('\n', text.strip())

    paragraphs_per_part = math.ceil(len(paragraphs) / (num_parts))

    parts = []
    current_part = ""

    for i, paragraph in enumerate(paragraphs):
        current_part += paragraph + "\n"
        if (i + 1) % paragraphs_per_part == 0 or (i + 1) == len(paragraphs):
            parts.append(current_part.strip())
            current_part = ""

    return parts

## Split text with overlap

In [15]:
def split_text_into_parts_with_overlap(text: str, num_parts: int):
    sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', text.strip())

    sentences_per_part = math.ceil(len(sentences) / num_parts)

    parts = []
    for i in range(num_parts):
        start_index = max(0, i * sentences_per_part - 1)
        end_index = min(len(sentences), (i + 1) * sentences_per_part + 1)

        part = " ".join(sentences[start_index:end_index])
        parts.append(part.strip())

    return parts

## Claude generate

In [16]:
def claude_generate_answer(model: str, prompt: str):
    response = client.messages.create(
        model=model,
        max_tokens=2000,
        temperature=0,
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": prompt
                    }
                ]
            }
        ]
    )

    return response

# Retrieve JSON from response

In [17]:
def get_json_from_response(text):
    json_match = re.search(r"<json_format>(.*?)</json_format>", text, re.DOTALL)
    if json_match:
        json_str = json_match.group(1).strip()
    else:
        markdown_match = re.search(r"```json(.*?)```", text, re.DOTALL)
        if markdown_match:
            json_str = markdown_match.group(1).strip()
        else:
            raise ValueError("JSON не найден ни в формате <json_format>, ни в формате Markdown.")

    try:
        return json.loads(json_str)
    except json.JSONDecodeError as e:
        raise ValueError(f"Ошибка декодирования JSON: {e}")

# Generate answers

In [18]:
text = read_pdf('../../books/zahar-berkut.pdf')
parts = split_text_into_parts(text, 10)
input_tokens = 0
output_tokens = 0

questions = []

for part in parts:
    prompt = get_gen_prompt(part)
    response = claude_generate_answer(model, prompt)
    input_tokens += response.usage.input_tokens
    output_tokens += response.usage.output_tokens
    json_part = get_json_from_response(response.content[0].text)
    questions.append(json_part)

    print(response.content[0].text)
    print("--------------------------------")
    print(json_part)
    print("================================")

with open('../../testing_data/claude-sonet-3-7-zahar-berkut.json', 'w', encoding='utf-8') as f:
    json.dump(questions, f, ensure_ascii=False, indent=4)

<question_development>
Let me identify key quotes from the excerpt that could be potential question sources:

1. "Дивний сон снився Захарові Беркутові. Здавалось йому, немов нині їхнє щорічне свято «Сторожа» і вся громада зібрана довкола каменя при вході до тухольської тіснини: дівчата з вінками, молодці з музикою, усі в празничних чистих строях."

2. "Оце він, найстарший за віком у громаді, перший наближається до святого каменя і починає молитися до нього. Якісь таємничі, тривожні, болючі чуття опановують його серце під час молитви; щось щемить у нього в глибині душі — і сам він не знає, що такого."

3. "Але слова не перестають пливти, темно робиться кругом, чорні хмари покривають небо, громи починають бити, блискавиці палахкотять та облітають увесь небозвід осліпляючим огнем, земля здригається — і разом, звільна перехиляючись, святий камінь рушається з місця та зі страшенним лускотом валиться на нього."

Relevant topics from the provided list:
- Topic 3: what the characters did in ce

In [19]:
print(input_tokens)
print(output_tokens)

18163
13674


In [20]:
print(response.content[0].text)

<question_development>
Let me identify key quotes from the excerpt:

1. "Твої тухольці обскочили нас у тій долині, завалили вихід. Розуміється, їхній опір тілько сміху варт, бо прецінь же вони не спинять нас."
2. "Тухольці — ціпкий народ: кого раз у руки зловлять, то вже не люблять пустити."
3. "Я думаю нині ще йти до твоїх тухольців для переговорів: хочу обіцяти їм тебе взамін за вільний прохід."
4. "Тухольці будуть битися до остатнього, щоб не пустити вас через гори."
5. "Говори щиро, по правді — се єдине чародійське слово."

Relevant topics from the provided list:
- Topic 3: what the characters did in certain situations
- Topic 4: what the characters did in certain situations, what they thought or felt
- Topic 5: the attitude of some characters to other characters

Potential questions:

Question 1:
Що боярин хотів запропонувати тухольцям під час переговорів?
a) Віддати їм золото взамін за вільний прохід (incorrect)
b) Віддати їм Максима взамін за вільний прохід (correct)
c) Залишити